In [2]:
import pandas as pd
import os
from ipni.tools import get_standard_column_names

In [30]:
folder = "/home/ceb/.biokb/ipni/data/unzipped/"
mapper = {"object": "str", "float64": "float", "int64": "int", "bool": "bool"}
lines = []

for file_name in os.listdir(folder):
    class_name = file_name.split(".")[0]
    lines.append(f"\n\nclass {class_name.capitalize()}(Base):")
    lines.append(f'    __tablename__ = Base._prefix + "{class_name.lower()}"\n')
    path = os.path.join(folder, file_name)
    df = pd.read_csv(path, sep="\t", low_memory=False)
    df.columns = get_standard_column_names(df.columns)
    for column in df.columns:
        props = []
        if df[column].is_unique:
            props.append("unique=True")

        data_type = mapper[str(df[column].dtype)]

        if data_type == "str":
            props = ["String(255)"] + props

        nullable = any(df[column].isnull())
        if nullable:
            data_type = f"Optional[{data_type}]"

        all_null = " # all null " if all(df[column].isnull()) else ""

        props_str = ", ".join([x for x in props])
        lines.append(
            f"    {column}: Mapped[{data_type}] = mapped_column({props_str}){all_null}"
        )

with open("models.py", mode="w") as fd:
    fd.write("\n".join(lines))

In [26]:
df[column].is_unique

False